In [3]:
from math import radians, cos, sin, asin, sqrt
import numpy as np

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
import pandas as pd
"""
Thanks stack overflow! lawl
"""

'\nThanks stack overflow! lawl\n'

In [4]:
def haversine(lonlat1, lonlat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, lon1 = lonlat1
    lat2, lon2 = lonlat2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [5]:
df = pd.read_json('./location.json')
df = df.transpose()
df.tail()

,lat,long
"Williamsport, PA",41.249329,-77.002767
"Woonsocket, RI",42.002876,-71.514784
"Wyandanch, NY",40.753988,-73.360398
"Ybor City, FL",27.960048,-82.441159
"Youngstown, OH",41.099780,-80.649519


In [6]:
distance_matrix = squareform(pdist(df, (lambda u,v: haversine(u,v))))

In [7]:
distance_matrix

array([[    0.        ,  2053.6453667 ,  1671.27879361, ...,
          962.37512952,  1601.88889549,   363.72446588],
       [ 2053.6453667 ,     0.        ,   438.16217178, ...,
         2968.74061109,  2420.1960345 ,  2361.66443128],
       [ 1671.27879361,   438.16217178,     0.        , ...,
         2558.40015348,  2001.31943123,  1958.77846337],
       ..., 
       [  962.37512952,  2968.74061109,  2558.40015348, ...,
            0.        ,  1646.59521151,   613.40612191],
       [ 1601.88889549,  2420.1960345 ,  2001.31943123, ...,
         1646.59521151,     0.        ,  1470.16321697],
       [  363.72446588,  2361.66443128,  1958.77846337, ...,
          613.40612191,  1470.16321697,     0.        ]])

In [10]:
db = DBSCAN(eps=50, min_samples=1, metric='precomputed')

In [11]:
y_db = db.fit_predict(distance_matrix)
len(np.unique(y_db))


150

In [ ]:
df['cluster'] = y_db
plt.scatter(df['long'], df['lat'], c=df['cluster'])
plt.show()

In [42]:
df.transpose()

,"Albion, MI","Albuquerque, NM","Amarillo, TX","Anchorage, AK","Anderson County, TX","Anniston, AL","Arlington, VA","Aroostook County, ME","Atlanta, GA","Aurora, IL",...,"Warrensville Heights, OH","Washington, D.C.","West Frankfort, IL","West Palm Beach, FL","Wichita, KS","Williamsport, PA","Woonsocket, RI","Wyandanch, NY","Ybor City, FL","Youngstown, OH"
lat,42.243097,35.084103,35.207219,61.216313,31.781924,33.695381,38.890396,46.684796,33.749099,41.760585,...,41.435350,36.298852,37.897827,26.715342,37.692236,41.249329,42.002876,40.753988,27.960048,41.099780
long,-84.753030,-106.650985,-101.833825,-149.894852,-95.625820,-85.839842,-77.084158,-68.470113,-84.390185,-88.320071,...,-81.514824,-82.359193,-88.931458,-80.053375,-97.337545,-77.002767,-71.514784,-73.360398,-82.441159,-80.649519
cluster,-1.000000,0.000000,-1.000000,-1.000000,1.000000,2.000000,3.000000,-1.000000,2.000000,4.000000,...,17.000000,13.000000,-1.000000,24.000000,-1.000000,-1.000000,11.000000,12.000000,19.000000,17.000000


In [43]:
df.to_dict(orient='index')
import json
with open('location_clustered.json', 'w') as fp:
    json.dump(df.to_dict(orient='index'), fp)